In [1]:
import chromadb

from chromadb.utils import embedding_functions

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")


client = chromadb.Client()
# client.heartbeat()
# client.reset()
collection = client.get_or_create_collection("test-where-list", embedding_function=sentence_transformer_ef)
collection.add(documents=["Article by john", "Article by Jack", "Article by Jill"],
               metadatas=[{"author": "john"}, {"author": "jack"}, {"author": "jill"}], ids=["1", "2", "3"])

query = ["Give me articles by john"]
res = collection.query(query_texts=query,where={'author': {'$in': ['john', 'jill']}}, n_results=10)
print(res)

res_get = collection.get(where={'author': {'$in': ['john', 'jill']}})
print(res_get)


Number of requested results 10 is greater than number of elements in index 3, updating n_results = 3


{'ids': [['1', '3']], 'distances': [[0.28824201226234436, 1.017508625984192]], 'metadatas': [[{'author': 'john'}, {'author': 'jill'}]], 'embeddings': None, 'documents': [['Article by john', 'Article by Jill']]}
{'ids': ['1', '3'], 'embeddings': None, 'metadatas': [{'author': 'john'}, {'author': 'jill'}], 'documents': ['Article by john', 'Article by Jill']}


# Interactions with existing Where operators

In [2]:
collection.upsert(documents=["Article by john", "Article by Jack", "Article by Jill"],
               metadatas=[{"author": "john","article_type":"blog"}, {"author": "jack","article_type":"social"}, {"author": "jill","article_type":"paper"}], ids=["1", "2", "3"])

collection.query(query_texts=query,where={"$and":[{"author": {'$in': ['john', 'jill']}},{"article_type":{"$eq":"blog"}}]}, n_results=3)

{'ids': [['1']],
 'distances': [[0.28824201226234436]],
 'metadatas': [[{'article_type': 'blog', 'author': 'john'}]],
 'embeddings': None,
 'documents': [['Article by john']]}

In [3]:
collection.query(query_texts=query,where={"$or":[{"author": {'$in': ['john']}},{"article_type":{"$in":["paper"]}}]}, n_results=3)

{'ids': [['1', '3']],
 'distances': [[0.28824201226234436, 1.017508625984192]],
 'metadatas': [[{'article_type': 'blog', 'author': 'john'},
   {'article_type': 'paper', 'author': 'jill'}]],
 'embeddings': None,
 'documents': [['Article by john', 'Article by Jill']]}